In [2]:
import tensorflow as tf

x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x*x*y+y+2
f2=x*x+y
g=tf.global_variables_initializer()

f.graph is tf.get_default_graph()# every node is added to default graph
graph=tf.Graph()#Creating a new graph and adding the nodes to the same.
with graph.as_default():
    x2=tf.Variable(5)
x2.graph is graph

True

In [3]:
with tf.Session() as sess:
    g.run()
    result=f.eval()
    f_val,f2_val=sess.run([f,f2])
    print(f_val,f2_val)

42 13


In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m, n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]

x=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
xt=tf.transpose(x)
theta=tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)),tf.matmul(xt,y))

with tf.Session() as sess:
    theta_value=theta.eval()
print(theta_value)

[[-3.7421875e+01]
 [ 4.3584442e-01]
 [ 9.3460083e-03]
 [-1.0673523e-01]
 [ 6.4443970e-01]
 [-4.2263418e-06]
 [-3.7744045e-03]
 [-4.2620850e-01]
 [-4.4000244e-01]]


In [5]:
from sklearn.preprocessing import StandardScaler

tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01
scaler=StandardScaler()
housing_data_plus_bias_scaled=scaler.fit_transform(housing_data_plus_bias)#scaling important for gradient descent
x=tf.constant(housing_data_plus_bias_scaled,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
xt=tf.transpose(x)
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")

y_pred=tf.matmul(x,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=(2/m)*tf.matmul(xt,error)
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("epoch: ",epoch,"mse:",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    print(best_theta)

epoch:  0 mse: 8.659345
epoch:  100 mse: 5.035349
epoch:  200 mse: 4.922659
epoch:  300 mse: 4.8889575
epoch:  400 mse: 4.867003
epoch:  500 mse: 4.8510027
epoch:  600 mse: 4.8391786
epoch:  700 mse: 4.830402
epoch:  800 mse: 4.8238635
epoch:  900 mse: 4.818972
[[-0.45982814]
 [ 0.8637171 ]
 [ 0.15735266]
 [-0.269707  ]
 [ 0.28265893]
 [ 0.00909606]
 [-0.04331845]
 [-0.5864663 ]
 [-0.5581758 ]]


In [6]:
A=tf.placeholder(tf.float32,shape=(None,None))
f=A+3
with tf.Session() as sess:
    val=f.eval(feed_dict={A:[[1],[2]]})
print(val)

[[4.]
 [5.]]


In [7]:
tf.reset_default_graph()
from sklearn.preprocessing import StandardScaler
from datetime import datetime

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run_{}/".format(root_logdir,now)

tf.reset_default_graph()
n_epochs=10
learning_rate=0.01
scaler=StandardScaler()
housing_data_plus_bias_scaled=scaler.fit_transform(housing_data_plus_bias)#scaling important for gradient descent

x=tf.placeholder(tf.float32,shape=(None,n+1),name="x")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = housing_data_plus_bias_scaled[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

batch_size=100
n_batches=int(np.ceil(m/batch_size))

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")

y_pred=tf.matmul(x,theta,name="predictions")

with tf.name_scope("loss") as scope:
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error),name="mse")

optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()
#saver
saver=tf.train.Saver()
#summary
mse_summary=tf.summary.scalar("MSE",mse)
fileWriter=tf.summary.FileWriter(logdir,tf.get_default_graph())


with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={x: x_batch,y: y_batch})  
            if batch_index%10 == 0:
                summary_str=mse_summary.eval(feed_dict={x:housing_data_plus_bias_scaled,y:housing.target.reshape(-1,1)})
                step=epoch*n_batches+batch_index
                fileWriter.add_summary(summary_str,step)
        if epoch%100 == 0:
            save_path=saver.save(sess,"/tmp/mymodel.ckpt")
    best_theta=theta.eval()
    save_path=saver.save(sess,"/tmp/mymodel.ckpt")
    print(best_theta)
with tf.Session() as sess:
    saver.restore(sess,"/tmp/mymodel.ckpt")
    print(mse.eval(feed_dict={x:housing_data_plus_bias_scaled,y:housing.target.reshape(-1,1)}))
fileWriter.close()

[[-0.75101876]
 [ 0.8250937 ]
 [ 0.11623652]
 [-0.21509653]
 [ 0.3208811 ]
 [-0.00621601]
 [ 0.00419882]
 [-0.85742074]
 [-0.830659  ]]
INFO:tensorflow:Restoring parameters from /tmp/mymodel.ckpt
4.809901


In [8]:
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run_{}/".format(root_logdir,now)

tf.reset_default_graph()
def relu(X):
    with tf.name_scope("relu") as scope:
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name="weights")
        b=tf.constant(0.0,name="bias")
        z=tf.add(tf.matmul(X,w),b)
        return tf.maximum(0.0,z,name="relu_result")
    
n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name="output")
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

In [16]:
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run_{}/".format(root_logdir,now)

tf.reset_default_graph()
def relu(X):
    with tf.name_scope("relu") as scope:
        with tf.variable_scope("shared",reuse=True) as scope:
            threshold=tf.get_variable("threshold")
            dummy=tf.constant(0.0,name="dummy")
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name="weights")
        b=tf.constant(0.0,name="bias")
        z=tf.add(tf.matmul(X,w),b)
        return tf.maximum(0.0,z,name="relu_result")
    
n_features=3
with tf.variable_scope("shared") as scope:
    threshold=tf.get_variable("threshold",shape=[1],initializer=tf.constant_initializer(0.0))
X=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name="output")
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())